# NOTES
**1. Year_Factor:**
* might be relevant b/c weather observations might change —> rising global temp each year
* buildings with weather observations done in least recent years might seem MORE energy efficient
* perhaps, could introduce a stabilizing factor for rising temp?
* pretty advanced, leave for the end:) for now, drop Year_Factor b/c in df_test, only one value —> most recent year

**2. facility_type:**
* 60 different values, NO idea how to deal with this for now??
* 

**3. TO DO:**
* Think about how to use facility_type
* Review all the columns with missing/NaN values and do research as to how we might deal with them
* Visualizations for EDA

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import matplotlib
import matplotlib.pyplot     as plt
import seaborn               as sns
import statsmodels.api as sm
%matplotlib inline
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# SHAP is not optimized for matplotlib 3.5.1
#!pip install matplotlib==3.4.3

<a id="section-one"></a>
# Goal

Our goal is to predict the energy consumption of a building using various characteristics of the building's composition and location as well as climate and weather variables.

<a id="section-two"></a>
# Data
The WiDS Datathon 2022 focuses on a prediction task involving roughly 100k observations of building energy usage records collected over 7 years and a number of states within the United States. The dataset consists of building characteristics (e.g. floor area, facility type etc), weather data for the location of the building (e.g. annual average temperature, annual total precipitation etc) as well as the energy usage for the building and the given year, measured as Site Energy Usage Intensity (Site EUI). Each row in the data corresponds to the a single building observed in a given year. Your task is to predict the Site EUI for each row, given the characteristics of the building and the weather data for the location of the building.


### Covariates

- `id`: building id
- `Year_Factor`: anonymized year in which the weather and energy usage factors were observed
- `State_Factor`: anonymized state in which the building is located
- `building_class`: building classification
- `facility_type`: building usage type
- `floor_area`: floor area (in square feet) of the building
- `year_built`: year in which the building was constructed
- `energy_star_rating`: the energy star rating of the building
- `ELEVATION`: elevation of the building location
- `january_min_temp`: minimum temperature in January (in Fahrenheit) at the location of the building
- `january_avg_temp`: average temperature in January (in Fahrenheit) at the location of the building
- `january_max_temp`: maximum temperature in January (in Fahrenheit) at the location of the building
- `cooling_degree_days`: cooling degree day for a given day is the number of degrees where the daily average temperature exceeds 65 degrees Fahrenheit. Each month is summed to produce an annual total at the location of the building.
- `heating_degree_days`: heating degree day for a given day is the number of degrees where the daily average temperature falls under 65 degrees Fahrenheit. Each month is summed to produce an annual total at the location of the building.
- `precipitation_inches`: annual precipitation in inches at the location of the building
- `snowfall_inches`: annual snowfall in inches at the location of the building
- `snowdepth_inches`: annual snow depth in inches at the location of the building
- `avg_temp`: average temperature over a year at the location of the building
- `days_below_30F`: total number of days below 30 degrees Fahrenheit at the location of the building
- `days_below_20F`: total number of days below 20 degrees Fahrenheit at the location of the building
- `days_below_10F`: total number of days below 10 degrees Fahrenheit at the location of the building
- `days_below_0F`: total number of days below 0 degrees Fahrenheit at the location of the building
- `days_above_80F`: total number of days above 80 degrees Fahrenheit at the location of the building
- `days_above_90F`: total number of days above 90 degrees Fahrenheit at the location of the building
- `days_above_100F`: total number of days above 100 degrees Fahrenheit at the location of the building
- `days_above_110F`: total number of days above 110 degrees Fahrenheit at the location of the building
- `direction_max_wind_speed`: wind direction for maximum wind speed at the location of the building. Given in 360-degree compass point directions (e.g. 360 = north, 180 = south, etc.).
- `direction_peak_wind_speed`: wind direction for peak wind gust speed at the location of the building. Given in 360-degree compass point directions (e.g. 360 = north, 180 = south, etc.).
- `max_wind_speed`: maximum wind speed at the location of the building
- `days_with_fog`: number of days with fog at the location of the building

### Target

- `site_eui`: Site Energy Usage Intensity is the amount of heat and electricity consumed by a building as reflected in utility bills

<a id="section-three"></a>
# Downloading Datasets

In [ ]:
df_train = pd.read_csv("/kaggle/input/widsdatathon2022/train.csv")

From above, we can see that there are 75757 entries, and there exist null values in `year_built`, `energy_star_rating`, `direction_max_wind_speed`, `direction_peak_wind_speed`, `max_wind_speed`, and `days_with_fog`. In fact, there are quite a large number of rows containing null columns, so dropping rows with null values may not seem like a good strategy.

In [ ]:
df_test = pd.read_csv("/kaggle/input/widsdatathon2022/test.csv")

### Here we are looking at the basic statistics for our training dataset df_train:

In [ ]:
# Look at the basic statistics 
df_train.describe().T.style

## Preliminary Data Cleaning

### Feature Engineering:

### building_type variable
* For both df_train and df_test, we have identified two unique values of this categorical variable: Residential and Commercial
* We transformed this variable into a binary numerical one using simple encoding:
 * Residential: 0, Commercial: 1

In [ ]:
'''
# Encoding the Building Type:
# 0 — Residential, 1 — Commercial

# df_test
df_test.building_class.unique()
building_type = {'Residential' : 0, 'Commercial' : 1}
df_test.building_class = [building_type[item] for item in df_test.building_class]

# df_train
df_train.building_class.unique()
building_type = {'Residential' : 0, 'Commercial' : 1}
df_train.building_class = [building_type[item] for item in df_train.building_class]
'''

### state_factor variable
* In our df_test dataset, we identified 6 different values of the state_factor parameter: ['State_1' 'State_2' 'State_4' 'State_8' 'State_11' 'State_10']
* On the other hand, the df_train dataset contains 7 unique values: ['State_1' 'State_2' 'State_4' 'State_6' 'State_8' 'State_10' 'State_11']. Thus, there is one additional value
* We decided to incorporate the parameter into our modeling since there is a 6/7 overlap in the state_factor values between the training and testing dataset


In [ ]:
'''
print(df_test["State_Factor"].unique())
print(df_train["State_Factor"].unique())
'''

In [ ]:
'''
# Encoding the State Factor:
# One-hot encoding for 7 different anonymous states

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


# Creating instance of one-hot-encoder
encoding = OneHotEncoder(handle_unknown='ignore')

# df_test
enc_df = pd.DataFrame(encoding.fit_transform(df_test[['State_Factor']]).toarray())
# merge with df_test on key values
df_test = df_test.join(enc_df)
df_test = df_test.drop('State_Factor', 1)
# Renaming the binary cols to get rid of int names
df_test = df_test.rename(columns={0: 'State_1', 1:'State_2', 2:'State_4', 3:'State_6', 4:'State_8', 5:'State_10', 6:'State_11'})
df_test["State_11"] = 0.0

# df_train
enc_df = pd.DataFrame(encoding.fit_transform(df_train[['State_Factor']]).toarray())
# merge with df_train on key values
df_train = df_train.join(enc_df)
df_train = df_train.drop('State_Factor', 1)
# Renaming the binary cols to get rid of int names
df_train = df_train.rename(columns={0: 'State_1', 1:'State_2', 2:'State_4', 3:'State_6', 4:'State_8', 5:'State_10', 6:'State_11'})
'''

### facility_type variable
We understand that facility_type variable is key to predicting the site_eui of the building. However, there are 60 unique values in this categorical variable for both df_train and df_test. Thus, we used different strategies to see how we can transform this feature to limit the dimensionality of our dataset and also remove any categorical variables.

In [ ]:
'''
len(df_train["facility_type"].unique())
len(df_test["facility_type"].unique())
'''

#### Method 1: Manually grouping facility_type into 7 thematic categories

In [ ]:
'''
# Manually grouping 
df_train["facility_type"].value_counts()



High = ["Multifamily_Uncategorized", "Office_Uncategorized", "Education_Other_classroom", "Lodging_Hotel", "Commercial_Other",
       "Education_College_or_university", "Education_Uncategorized", "Mixed_Use_Commercial_and_Residential", "Office_Medical_non_diagnostic",
       "Health_Care_Inpatient", "Industrial", "Parking_Garage", "Retail_Strip_shopping_mall", "Warehouse_Refrigerated", 
       "Laboratory", "Health_Care_Uncategorized", "Data_Center"]

Medium = ["Warehouse_Nonrefrigerated", "Retail_Uncategorized", "Nursing_Home", "Grocery_store_or_food_market ",
         "Public_Assembly_Library", "Public_Assembly_Other", "Service_Vehicle_service_repair_shop", "Food_Sales", 
         "Food_Service_Restaurant_or_cafeteria", "Public_Safety_Penitentiary", "Health_Care_Outpatient_Uncategorized"]

Low = ["2to4_Unit_Building", "5plus_Unit_Building", "Lodging_Dormitory_or_fraternity_sorority", "Warehouse_Distribution_or_Shipping_center",
      "Warehouse_Selfstorage", "Religious_worship", "Warehouse_Uncategorized", "Mixed_Use_Predominantly_Commercial", "Office_Bank_or_other_financial",
      "Public_Safety_Fire_or_police_station", "Retail_Enclosed_mall", "Public_Assembly_Entertainment_culture", 
      "Education_Preschool_or_daycare", "Commercial_Unknown", "Public_Assembly_Social_meeting", "Lodging_Other", "Retail_Vehicle_dealership_showroom",
      "Public_Assembly_Recreation", "Public_Assembly_Drama_theater", "Service_Uncategorized", "Health_Care_Outpatient_Clinic", 
      "Public_Assembly_Movie_Theater", "Public_Safety_Courthouse", "Public_Assembly_Uncategorized", "Office_Mixed_use",
      "Food_Service_Uncategorized", "Food_Service_Other", "Mixed_Use_Predominantly_Residential", "Public_Assembly_Stadium", 
      "Service_Drycleaning_or_Laundry", "Lodging_Uncategorized"]

df_train["Residential"] = df_train["facility_type"].isin(residential) * 1
df_train["Lodging"] = df_train["facility_type"].isin(lodging) * 1
df_train["Commercial"] = df_train["facility_type"].isin(commercial) * 1
df_train["Industrial"] = df_train["facility_type"].isin(industrial) * 1
df_train["Office_classroom"] = df_train["facility_type"].isin(office_classroom) * 1
df_train["Services"] = df_train["facility_type"].isin(services) * 1
df_train["Recreational"] = df_train["facility_type"].isin(recreational) * 1


df_test["Residential"] = df_train["facility_type"].isin(residential) * 1
df_test["Lodging"] = df_train["facility_type"].isin(lodging) * 1
df_test["Commercial"] = df_train["facility_type"].isin(commercial) * 1
df_test["Industrial"] = df_train["facility_type"].isin(industrial) * 1
df_test["Office_classroom"] = df_train["facility_type"].isin(office_classroom) * 1
df_test["Services"] = df_train["facility_type"].isin(services) * 1
df_test["Recreational"] = df_train["facility_type"].isin(recreational) * 1

def group_facility(ft):
    if ft in (High):
        return "High"
    if ft in (Medium):
        return "Medium"
    if ft in (Low):
        return "Low"
    else:
        return "None"


df_train["facility_groups"] = df_train["facility_type"].map(group_facility)
df_test["facility_groups"] = df_test["facility_type"].map(group_facility)

df_train["facility_type"] = rename_facility(df_train)

df_train["High"] = df_train["facility_type"].isin(High) * 1
df_train["Medium"] = df_train["facility_type"].isin(Medium) * 1
df_train["Low"] = df_train["facility_type"].isin(Low) * 1

df_test["High"] = df_train["facility_type"].isin(High) * 1
df_test["Medium"] = df_train["facility_type"].isin(Medium) * 1
df_test["Low"] = df_train["facility_type"].isin(Low) * 1

df_train.drop("facility_type", 1)
df_test.drop("facility_type", 1)
'''


#### Method 2: Manually grouping facility_type into 3 categories based on site_eui
To perform this grouping, we plotted the site_eui for each unique value of the facility_type feature and identified the approximate maximum cluster. Then, based on these visual distributions, we divided all facility_type values into three groups: Low, Medium, and High

In [ ]:
'''
High = ["Multifamily_Uncategorized", "Office_Uncategorized", "Education_Other_classroom", "Lodging_Hotel", "Commercial_Other",
       "Education_College_or_university", "Education_Uncategorized", "Mixed_Use_Commercial_and_Residential", "Office_Medical_non_diagnostic",
       "Health_Care_Inpatient", "Industrial", "Parking_Garage", "Retail_Strip_shopping_mall", "Warehouse_Refrigerated", 
       "Laboratory", "Health_Care_Uncategorized", "Data_Center"]

Medium = ["Warehouse_Nonrefrigerated", "Retail_Uncategorized", "Nursing_Home", "Grocery_store_or_food_market ",
         "Public_Assembly_Library", "Public_Assembly_Other", "Service_Vehicle_service_repair_shop", "Food_Sales", 
         "Food_Service_Restaurant_or_cafeteria", "Public_Safety_Penitentiary", "Health_Care_Outpatient_Uncategorized"]

Low = ["2to4_Unit_Building", "5plus_Unit_Building", "Lodging_Dormitory_or_fraternity_sorority", "Warehouse_Distribution_or_Shipping_center",
      "Warehouse_Selfstorage", "Religious_worship", "Warehouse_Uncategorized", "Mixed_Use_Predominantly_Commercial", "Office_Bank_or_other_financial",
      "Public_Safety_Fire_or_police_station", "Retail_Enclosed_mall", "Public_Assembly_Entertainment_culture", 
      "Education_Preschool_or_daycare", "Commercial_Unknown", "Public_Assembly_Social_meeting", "Lodging_Other", "Retail_Vehicle_dealership_showroom",
      "Public_Assembly_Recreation", "Public_Assembly_Drama_theater", "Service_Uncategorized", "Health_Care_Outpatient_Clinic", 
      "Public_Assembly_Movie_Theater", "Public_Safety_Courthouse", "Public_Assembly_Uncategorized", "Office_Mixed_use",
      "Food_Service_Uncategorized", "Food_Service_Other", "Mixed_Use_Predominantly_Residential", "Public_Assembly_Stadium", 
      "Service_Drycleaning_or_Laundry", "Lodging_Uncategorized"]


df_train["High"] = df_train["facility_type"].isin(High) * 1
df_train["Medium"] = df_train["facility_type"].isin(Medium) * 1
df_train["Low"] = df_train["facility_type"].isin(Low) * 1

df_test["High"] = df_train["facility_type"].isin(High) * 1
df_test["Medium"] = df_train["facility_type"].isin(Medium) * 1
df_test["Low"] = df_train["facility_type"].isin(Low) * 1
'''

#### Method 3: Manually grouping facility_type into 5 categories based on energy_star_rating
1. First, df_train was grouped by facility_type
2. Then, for each category, the median value of the energy_star_rating is calculated. We decided to use the median since there are large, non-uniform distribution among of energy_star_rating each facility_type
3. Finally, based on the documentation found on the [Energy Star website](http://www.energystar.gov/buildings/benchmark/understand_metrics/how_score_calculated), we created a feature called facility_energy_star_rating. This feature has 5 different values: Very Low, Low, Medium, Top, and NaN. It reflects the median energy_star_rating of the facility_type that the building belongs to. The distribution is as follows:
 * Very Low: median energy_star_rating below 25,
 * Low: median energy_star rating between 25 and 50,
 * Medium: median energy_star rating between 50 and 75,
 * Top: median energy_star rating above 75,
 * If the median could not be calculated, the value is marked as NaN

In [ ]:
'''
# Create a dictionary of facility_type and corresponding median energy_star_rating
medians = {}
# Iterate through all unique values of facility_type
for facility_type in df_train['facility_type'].unique():
    # For each facility type, identify the value of mediam energy_star_rating
    medians[facility_type] = df_train[df_train['facility_type'] == facility_type]['energy_star_rating'].median()
    
# Create a new column in df_train and df_test
df_train['medians'] = [medians[row] for row in df_train['facility_type']]
# Create a new column in df_train
df_test['medians'] = [medians[row] for row in df_test['facility_type']]
'''

In [ ]:
'''
# Now, let's create a function to assign categorical values of Very Low, Low, Medium, High, NaN
# based on the value of the energy_star_rating median
def rank_median(row):
    if row['medians'] < 25:
        return 'Very Low'
    elif (row['medians'] >= 25) and (row['medians'] < 50):
        return 'Low'
    elif (row['medians'] >= 50) and (row['medians'] < 75):
        return 'Medium'
    elif row['medians'] >= 75:
        return 'Top'
    else:
        return 'NaN'
'''

In [ ]:
'''
# Create a new categorical column and drop original metrics, 'facility_type' and 'energy_star_rating'
df_train['facility_energy_star_rating'] = df_train.apply(lambda row: rank_median(row), axis=1)
df_train = df_train.drop(['facility_type', 'energy_star_rating', 'medians'], 1)

df_test['facility_energy_star_rating'] = df_test.apply(lambda row: rank_median(row), axis=1)
df_test = df_test.drop(['facility_type', 'energy_star_rating', 'medians'], 1)
'''

In [ ]:
'''
plt.figure(figsize=(10,10))
df_train["New"] = df_train["facility_type"] == "Lodging_Uncategorized"
plt.scatter(df_train["New"].astype(str), df_train["site_eui"])
'''

In [ ]:
'''
def rename_facility(df):
    names = df.facility_type.unique()
    cols = {}
    for i in range(len(names)):
        cols[i] = names[i]
    return cols
'''

In [ ]:
'''
# # Encode facility type
# solution for now to just automate OHE with 60 types
# but still looking into making that size a bit smaller

enc_df = pd.DataFrame(encoding.fit_transform(df_test[['facility_type']]).toarray())
#merge
df_test = df_test.join(enc_df)
#rename
names = rename_facility(df_test)
df_test = df_test.rename(columns = names)
#drop original
df_test = df_test.drop('facility_type', 1)

enc_df = pd.DataFrame(encoding.fit_transform(df_train[['facility_type']]).toarray())
#merge
df_train = df_train.join(enc_df)
#rename
names = rename_facility(df_train)
df_train = df_train.rename(columns = names)
#drop original
df_train = df_train.drop('facility_type', 1)
'''

### year_factor and id
* **year_factor:**
 * We discovered that there is only a single unique value of year_factor in df_test
 * This means that we cannot train our model on this particular feature
 * We decided to remove it entirely from both of our datasets

* **id:**
 * We decided to remove this feature since it's simply a list of consecutive integers that is only relevant during the submission process

In [ ]:
df_train = df_train.drop(['Year_Factor', 'id'], 1)
df_test = df_test.drop(['Year_Factor', 'id'], 1)

## Examining Variables and Relations Between Them

**Histogram 1.** Prevailing majority of the buildings in df_train have Site EUI of 25-50. There are a lot of outliers with very high Site EUI, for up to 997 

In [ ]:
#df_train["site_eui"].plot.hist(title="Site EUI", bins=20)

**Histogram 2.** 0 — Residential, 1 — Commercial. There are more residential buildings in the dataset

In [ ]:
#df_train["building_class"].plot.hist(title="Building Class", bins=20)

In [ ]:
#plt.figure(figsize=(50,20))
#plt.scatter(df_train["facility_type"].astype(str), df_train["site_eui"])

In [ ]:
#sns.boxplot(x = "building_class", y='site_eui', data = df_train)
#plt.title("Boxplot of Building Class vs. Site EUI")

In [ ]:
# Correlation for Building Features

# training data

#cols = ['building_class', 'floor_area', 'year_built', 'energy_star_rating', 'site_eui']
#df_train[cols].corr(method='pearson')

In [ ]:
#sns.heatmap(df_train[cols].corr(method='pearson'));

In [ ]:
# test data 

#cols = ['building_class', 'floor_area', 'year_built', 'energy_star_rating']
#df_test[cols].corr(method='pearson')

In [ ]:
#sns.heatmap(df_test[cols].corr(method='pearson'));

## Addressing Null/Missing Values

In [ ]:
'''
def missing_statistics(df):    
    statitics = pd.DataFrame(df.isnull().sum()).reset_index()
    statitics.columns=['COLUMN NAME',"MISSING VALUES"]
    statitics['TOTAL ROWS'] = df.shape[0]
    statitics['% MISSING'] = round((statitics['MISSING VALUES']/statitics['TOTAL ROWS'])*100,2)
    return statitics
'''

In [ ]:
'''
#code copied from https://www.kaggle.com/shrutisaxena/wids2022-starter-code

# year_built: replace with current year
df_train['year_built'] = df_train['year_built'].replace(np.nan, 2022)
# replacing rest of the values with mean
df_train['energy_star_rating'] = df_train['energy_star_rating'].replace(np.nan,df_train['energy_star_rating'].mean())
df_train['direction_max_wind_speed'] = df_train['direction_max_wind_speed'].replace(np.nan,df_train['direction_max_wind_speed'].mean())
df_train['direction_peak_wind_speed'] = df_train['direction_peak_wind_speed'].replace(np.nan,df_train['direction_peak_wind_speed'].mean())
df_train['max_wind_speed'] = df_train['max_wind_speed'].replace(np.nan,df_train['max_wind_speed'].mean())
df_train['days_with_fog'] = df_train['days_with_fog'].replace(np.nan,df_train['days_with_fog'].mean())

##for testdata
# year_built: replace with current year
df_test['year_built'] = df_test['year_built'].replace(np.nan, 1980)
# replacing rest of the values with mean
df_test['energy_star_rating'] = df_test['energy_star_rating'].replace(np.nan,df_test['energy_star_rating'].mean())
df_test['direction_max_wind_speed']= df_test['direction_max_wind_speed'].replace(np.nan,df_test['direction_max_wind_speed'].mean())
df_test['direction_peak_wind_speed']= df_test['direction_peak_wind_speed'].replace(np.nan,df_test['direction_peak_wind_speed'].mean())
df_test['max_wind_speed'] = df_test['max_wind_speed'].replace(np.nan,df_test['max_wind_speed'].mean())
df_test['days_with_fog'] = df_test['days_with_fog'].replace(np.nan,df_test['days_with_fog'].mean())
'''

In [ ]:
'''
# Use data imputer instead of mean to fill in null values

from sklearn.impute import SimpleImputer
null_col=['energy_star_rating','direction_max_wind_speed','direction_peak_wind_speed','max_wind_speed','days_with_fog']
imputer = SimpleImputer()
imputer.fit(df_train[null_col])
df_train_transformed = imputer.transform(df_train[null_col])
df_train[null_col] = pd.DataFrame(df_train_transformed)
test_data_transformed = imputer.transform(df_test[null_col])
df_test[null_col] = pd.DataFrame(test_data_transformed)
'''

# EDA and Visualizations

In [ ]:
# Visualize site_eui for training data

#sns.distplot(df_train['site_eui'], kde = True, hist = True, color = "green")

In [ ]:
'''
# Visualize feature distributions 

cols=['floor_area','year_built', 'energy_star_rating',
       'ELEVATION',  'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inches',
       'snowdepth_inches', 'avg_temp', 'days_below_30F', 'days_below_20F',
       'days_below_10F', 'days_below_0F', 'days_above_80F', 'days_above_90F',
       'days_above_100F', 'days_above_110F', 'direction_max_wind_speed',
       'direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog', 'State_1', 'State_2',
       'State_4', 'State_6', 'State_8', 'State_10', 'State_11']

# for col in cols:
#     plt.figure(figsize=(15, 2))
#     sns.distplot(df_train[col],kde = True, hist = True, color = "green")
#     plt.title(col+" distribution")
#     plt.show()
'''

## Weather Variables

In [ ]:
'''
weather_df=['site_eui', 'ELEVATION', 'january_min_temp',
       'january_avg_temp', 'january_max_temp', 'february_min_temp',
       'february_avg_temp', 'february_max_temp', 'march_min_temp',
       'march_avg_temp', 'march_max_temp', 'april_min_temp', 'april_avg_temp',
       'april_max_temp', 'may_min_temp', 'may_avg_temp', 'may_max_temp',
       'june_min_temp', 'june_avg_temp', 'june_max_temp', 'july_min_temp',
       'july_avg_temp', 'july_max_temp', 'august_min_temp', 'august_avg_temp',
       'august_max_temp', 'september_min_temp', 'september_avg_temp',
       'september_max_temp', 'october_min_temp', 'october_avg_temp',
       'october_max_temp', 'november_min_temp', 'november_avg_temp',
       'november_max_temp', 'december_min_temp', 'december_avg_temp',
       'december_max_temp', 'cooling_degree_days', 'heating_degree_days',
       'precipitation_inches', 'snowfall_inches', 'snowdepth_inches',
       'avg_temp', 'days_below_30F', 'days_below_20F', 'days_below_10F',
       'days_below_0F', 'days_above_80F', 'days_above_90F', 'days_above_100F',
       'days_above_110F', 'direction_max_wind_speed',
       'direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog',]
'''

In [ ]:
# Examining correlations between weather variables
#corrm = df_train[weather_df].corr()
#corrm

In [ ]:
# Heatmap correspoding to the weather correlation matrix
#plt.figure(figsize=(20, 10))
#sns.heatmap(corrm)

### Positive Correlations between site_eui and Weather Variables

In [ ]:
#positive_f = corrm["site_eui"] > 0
#corrm["site_eui"][positive_f].sort_values(ascending = False).to_frame()

### Negative Correlations between site_eui and Weather Variables

In [ ]:
#negative_f = corrm["site_eui"] < 0
#corrm["site_eui"][negative_f].sort_values(ascending = False).to_frame()

### Pairplots to Examine Relations between site_eui and Weather Var-s

In [ ]:
# for col in cols:
#     plt.figure(figsize=(20, 10))
#     sns.pairplot(data=df_train,
#                  y_vars=['site_eui'],
#                  x_vars=[col])
#     plt.title('Pairplot of ' + col + ' vs. Site EUI')
#     plt.show()

## Manually Dropping Weather Var-s of Correlation > 0.9

In [ ]:
# january_avg_temp
#positive_f = corrm["january_avg_temp"] > 0
#corrm["january_avg_temp"][positive_f].sort_values(ascending = False).to_frame()

In [ ]:
'''
weather_var_drop1 = ['january_min_temp',
                    'february_avg_temp',
                    'march_min_temp']
'''

In [ ]:
# february_avg_temp —> drop itself b/c correlation with january_avg_temp	0.911228
#positive_f = corrm["february_avg_temp"] > 0
#corrm["february_avg_temp"][positive_f].sort_values(ascending = False).to_frame()

In [ ]:
#weather_var_drop2 = ['march_min_temp']

In [ ]:
# july_avg_temp
#positive_f = corrm["july_avg_temp"] > 0
#corrm["july_avg_temp"][positive_f].sort_values(ascending = False).to_frame()

In [ ]:
# cooling_degree_days
#positive_f = corrm["cooling_degree_days"] > 0
#corrm["cooling_degree_days"][positive_f].sort_values(ascending = False).to_frame()

In [ ]:
'''
weather_var_drop3 = ['august_avg_temp',
                     'july_avg_temp']
'''

In [ ]:
# december_avg_temp
#positive_f = corrm["december_avg_temp"] > 0
#corrm["december_avg_temp"][positive_f].sort_values(ascending = False).to_frame()

In [ ]:
#weather_var_drop4 = ['december_min_temp']

In [ ]:
# days_below_20F
#positive_f = corrm["days_below_20F"] > 0
#corrm["days_below_20F"][positive_f].sort_values(ascending = False).to_frame()

In [ ]:
#weather_var_drop5 = ['days_below_10F']

In [ ]:
# max_wind_speed
#positive_f = corrm["max_wind_speed"] > 0
#corrm["max_wind_speed"][positive_f].sort_values(ascending = False).to_frame()

In [ ]:
'''
weather_var_drop6 = ['direction_max_wind_speed',
                     'direction_peak_wind_speed']
'''

In [ ]:
#weather_var_drop = weather_var_drop1 + weather_var_drop2 + weather_var_drop3 + weather_var_drop4 + weather_var_drop5 + weather_var_drop6
#weather_var_drop

# Trying Out Our First Model :)

In [ ]:
'''
y_train = df_train['site_eui']
X_train = df_train.drop('site_eui', 1)
X_test = df_test
'''

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit_transform(X_train)

In [ ]:
# lin_reg_mod = LinearRegression()
# lin_reg_mod.fit(X_train, y_train)
# pred = lin_reg_mod.predict(X_test)

In [ ]:
# pred

In [ ]:
# # Calculating Root Mean Squared Error (RMSE) on train data (will be different than value for test data)

# MSE = np.square(np.subtract(y_train,lin_reg_mod.predict(X_train))).mean() 
# RMSE = math.sqrt(MSE)
# print("Root Mean Squared Error:\n")
# print(RMSE)

# GLM

In [ ]:
# import statsmodels.api as sm
# from scipy import stats

# # data = df_train

# # glm_binom = sm.GLM(data.endog(), sm.add_constant(data.exog, prepend=False), family=sm.families.Binomial())
# # res = glm_binom.fit()
# # print(res.summary())

# exog, endog = sm.add_constant(X_train), y_train
# mod = sm.GLM(endog, exog,
#              family=sm.families.Poisson(link=sm.families.links.log))
# res = mod.fit()

In [ ]:
# pred = res.predict(sm.add_constant(X_test))
# train_predi = res.predict(exog)

In [ ]:
# MSE = np.square(np.subtract(y_train,train_predi)).mean() 
# RMSE = math.sqrt(MSE)
# print("Root Mean Squared Error:\n")
# print(RMSE)

# Random Forest

In [ ]:
'''
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=100, random_state=0)
regressor.fit(X_train, y_train)
'''

In [ ]:
'''
y_pred = regressor.predict(X_train)
pred = regressor.predict(X_test)
'''

In [ ]:
'''
MSE = np.square(np.subtract(y_train,y_pred)).mean() 
RMSE = math.sqrt(MSE)
print("Root Mean Squared Error:\n")
print(RMSE)
# Submission: 52.something with n = 20
# Sumbission2: 47.719 with n = 100
'''

# Gradient Boosting

In [ ]:
'''
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
reg = BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse'),
                       n_estimators=100, random_state=0)
reg.fit(X_train, y_train)
'''

In [ ]:
'''
y_pred = reg.predict(X_train)
pred = reg.predict(X_test)
'''

In [ ]:
'''
MSE = np.square(np.subtract(y_train,y_pred)).mean() 
RMSE = math.sqrt(MSE)
print("Root Mean Squared Error:\n")
print(RMSE)
'''

## Trying LGBM

In [ ]:
y = df_train.pop("site_eui")

In [ ]:
data = pd.concat([df_train, df_test])
data = pd.get_dummies(data)
train_df = data.iloc[:len(df_train)]
test_df = data.iloc[len(df_train):]

In [ ]:
# LGBM

import lightgbm as lgbm

# n_estimators = number of trees
# types of boosting:
# ‘gbdt‘: Gradient Boosting Decision Tree (GDBT)
# ‘dart‘: Dropouts meet Multiple Additive Regression Trees (DART)
# ‘goss‘: Gradient-based One-Side Sampling (GOSS)


# 0.03855003139619445
# .03855
model = lgbm.LGBMRegressor(n_estimators = 2555, num_leaves = 407, objective='regression', metric='rmse', 
                           min_child_samples = 3, learning_rate = 0.00999, boosting_type = 'gbdt', 
                           colsample_bytree = 0.8052383153603662, 
                           reg_alpha = 0.0504937624635166, reg_lambda = 0.0009765625)

model.fit(train_df, y)
result = model.predict(test_df)

Finetune parameters after recommendation from AutoML: n_estimators, num_leaves, learning_rate

Best: 33.708 with n_estimators = 2555, num_leaves = 407, learning_rate = .00999

In [ ]:
sub = pd.read_csv("../input/widsdatathon2022/sample_solution.csv")
sub["site_eui"] = result
sub.to_csv("submission.csv", index=False)
sub.head()

## Trying Out Catboost

In [ ]:
from catboost import CatBoostRegressor
from sklearn import model_selection

In [ ]:
def get_model():
    cat_params = {
        'iterations': 30000, # 35000
        'learning_rate': 0.025,
        'od_wait': 1000,
        'depth': 8,
        'task_type' : 'GPU',
        'devices' : '0',
        'verbose' : 1000,
        "objective": "RMSE",
        "loss_function": "RMSE"
    }
    model =  CatBoostRegressor(**cat_params)
    return model

In [ ]:
kfold = model_selection.KFold(7, shuffle=True, random_state=2022)
models = []
train_with_k_fold = False
apply_psudo_labeling = False
if train_with_k_fold:
    for train_indices, val_indices in kfold.split(train_df):
        X_train = train_df.iloc[train_indices]
        y_train = y.iloc[train_indices]
       
        if apply_psudo_labeling:
            X_train = X_train.append(test_df)
            y_train = y_train.append(psudo_labels["site_eui"])
        X_val = train_df.iloc[val_indices]
        y_val = y.iloc[val_indices]
        model = get_model()
        model.fit(X_train, y_train, eval_set=(X_val, y_val))
        models.append(model)
else:
    model = get_model()
    if apply_psudo_labeling:
        train_df = train_df.append(test_df)
        y = y.append(psudo_labels["site_eui"])
    model.fit(train_df, y, plot = True) # early_stopping_rounds = int
    print("Best iteration: " + str(model.get_best_iteration()))
    models.append(model)



# Converting Predictions into a CSV File

In [ ]:
sub = pd.read_csv("../input/widsdatathon2022/sample_solution.csv")
sub["site_eui"] = pred
sub.to_csv("submission.csv", index = False)

In [ ]:
result = np.mean([model.predict(test_df) for model in models], axis=0)
sub = pd.read_csv("../input/widsdatathon2022/sample_solution.csv")
sub["site_eui"] = result
sub.to_csv("submission.csv", index=False)
sub.head()

### Performance of Catboost:
1. NO pre-processing, original dataset —> 35.821
2. With encoding for building_type —> 36.723
3. With encoding for building_type, state_factor —> 38.843